In [ ]:
import phd

import os
import numpy as np
import nengo
import nengo.utils.numpy as npext
from nengo_gui.ipython import IPythonViz

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('figure', figsize=(12, 3))

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Basic usage

In [ ]:
model = phd.sermo.Recognition()
model.trial.dt = 0.001
model.syllable.similarity_th = 0.83
model.syllable.scale = 0.66
freqs = [('ges-de-ccv', 'blaa', 1.2), ('ges-de-cv', 'tii', 1.8), ('ges-de-cvc', 'das', 1.0)]
for gdir, ges, freq in freqs:
    path = phd.ges_path(gdir, '%s.ges' % ges.lower())
    traj = phd.vtl.parse_ges(path).trajectory(dt=model.trial.dt)
    model.add_syllable(label=ges.upper(),
                       trajectory=traj)
    # Freq unused here
model.trial.trajectory = np.vstack([s.trajectory for s in model.syllables])

In [ ]:
net = model.build()
with net:
    p_traj = nengo.Probe(net.trajectory.output, synapse=0.01)
    p_dmps = [nengo.Probe(dmp.state, synapse=0.01) for dmp in net.syllables]
    p_mem = nengo.Probe(net.memory.output, synapse=0.01)
    p_class = nengo.Probe(net.classifier, synapse=0.01)
    p_resets = [nengo.Probe(dmp.reset, synapse=0.01) for dmp in net.syllables]

In [ ]:
sim = nengo.Simulator(net)
sim.run(sum(s.trajectory.shape[0] * model.trial.dt for s in model.syllables))

In [ ]:
labels = [label for _, label, _ in freqs]
t = sim.trange()
plt.figure()
plt.plot(t, sim.data[p_traj])
plt.xlim(right=t[-1])
for pr, label in zip(p_dmps, labels):
    plt.figure()
    plt.plot(t, sim.data[pr])
    plt.xlim(right=t[-1])
plt.figure()
plt.plot(t, nengo.spa.similarity(sim.data[p_mem], net.vocab, True))
plt.xlim(right=t[-1])
plt.legend(labels, loc='best')
plt.figure()
plt.plot(t, sim.data[p_class])
plt.xlim(right=t[-1]);

## Running an experiment

In [ ]:
model = phd.sermo.Recognition()
model.syllable.similarity_th = 0.8
model.syllable.scale = 0.72
expt = phd.experiments.RecognitionExperiment(model, n_syllables=5, sequence_len=2)
key = expt.run()
res = phd.experiments.RecognitionResult.load(key)

In [ ]:
print("Should be %s" % res.seq)
plt.plot(res.memory)
plt.legend(res.dmp_labels)

In [ ]:
plt.plot(res.dmps);

In [ ]:
print("accuracy: %f" % res.acc)
print("subst: %d, del: %d, ins: %d" % (res.n_sub, res.n_del, res.n_ins))
print("time diff: %f +/- %f" % (res.tdiff_mean, res.tdiff_var))
print("memory acc: %f" % res.memory_acc)

## Plotting experimental results

In [ ]:
def time_plots(key, x_label):
    df = phd.analysis.load_results(phd.experiments.RecognitionResult, [key])

    cmp_args = {'group_by': key,
                'x_label': x_label}

    plt.figure()
    phd.plots.timeseries(df,
                      columns=['acc'],
                      x_keys=['Accuracy'],
                      y_label='Accuracy',
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.timeseries(df,
                      columns=['n_sub', 'n_del', 'n_ins'],
                      x_keys=['Substitutions', 'Deletions', 'Insertions'],
                      y_label='Count',
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.timeseries(df,
                      columns=['tdiff_mean', 'tdiff_var'],
                      x_keys=['Timing mean', 'Timing variance'],
                      y_label='Timing',
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.timeseries(df,
                      columns=['memory_acc'],
                      x_keys=['Memory accuracy'],
                      y_label='Accuracy',
                      **cmp_args)
    plt.xlabel("")

def cmp_plots(key, x_label, hue_order):
    df = phd.analysis.load_results(phd.experiments.RecognitionResult, [key])

    cmp_args = {'group_by': key,
                'x_label': x_label,
                'hue_order': hue_order}

    plt.figure()
    phd.plots.compare(df,
                      columns=['acc'],
                      x_keys=['Accuracy'],
                      y_label='Accuracy',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.compare(df,
                      columns=['n_sub', 'n_del', 'n_ins'],
                      x_keys=['Substitutions', 'Deletions', 'Insertions'],
                      y_label='Count',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.compare(df,
                      columns=['tdiff_mean', 'tdiff_var'],
                      x_keys=['Timing mean', 'Timing variance'],
                      y_label='Timing',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")

    plt.figure()
    phd.plots.compare(df,
                      columns=['memory_acc'],
                      x_keys=['Memory accuracy'],
                      y_label='Accuracy',
                      plot_f=sns.barplot,
                      **cmp_args)
    plt.xlabel("")

In [ ]:
time_plots('scale', 'Scale')

In [ ]:
time_plots('similarity', 'Similarity threshold')

In [ ]:
time_plots('syllneurons', 'Syllable neurons')

In [ ]:
time_plots('freq', 'Syllable frequency')

In [ ]:
time_plots('n_syllables', 'Syllabary size')

In [ ]:
time_plots('sequence_len', 'Sequence length')

In [ ]:
cmp_plots('repeat', 'Repeat syllables', hue_order=['False', 'True'])

## Scaling

In [ ]:
def n_neurons(msg, model, n_syllables):
    net = model.build()
    nn = sum(e.n_neurons for e in net.all_ensembles)
    print("=== %s ===" % msg)

    inneurons = 80 * 48
    amneurons = ((n_syllables * 3) * 50)  # 50 is default for AM
    memneurons = model.cleanup.dimensions * model.memory.neurons_per_dimension
    # Note: depends on gestures per syllable.
    g_per_syll = np.mean([d.state.dimensions - 1 for d in net.syllables])
    print("g per syll: %.3f" % g_per_syll)
    syllneurons = n_syllables * (model.syllable.n_per_d * (g_per_syll + 2) + 20)
    clsneurons = 20

    allneurons = inneurons + amneurons + memneurons + syllneurons + clsneurons

    print("in: %d, spa: %d, syll: %d, cls %d" % (
        inneurons, amneurons + memneurons, syllneurons, clsneurons))

    print("Total: %d neurons" % allneurons)
    print("")

def add_syllables(model, n_syllables):
    model.trial.trajectory = np.zeros((2, 48))
    for p, f in zip(*phd.experiments.get_syllables(n_syllables, 1, 1)):
        tr = phd.vtl.parse_ges(p).trajectory(model.trial.dt)
        lbl = phd.experiments.path2label(p)
        model.add_syllable(label=lbl, freq=f, trajectory=tr)

model = phd.sermo.Recognition()
add_syllables(model, 3)
net = model.build()
n_neurons("Default configuration", model, 3)
model = phd.sermo.Recognition()
model.cleanup.dimensions = 256
add_syllables(model, 3)
net = model.build()
n_neurons("Conservative estimate", model, 2000)
model = phd.sermo.Recognition()
model.cleanup.dimensions = 512
add_syllables(model, 3)
net = model.build()
n_neurons("Generous estimate", model, 4000);